In [3]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_openai


In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    api_key="use_your_github_ai_api_key_here",
    base_url="https://models.github.ai/inference",
    model="gpt-4o-mini"
)

In [5]:
llm.invoke("hello")

AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 8, 'total_tokens': 18, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_f97eff32c5', 'finish_reason': 'stop', 'logprobs': None}, id='run-fa88f703-fea4-4514-908e-a4e3a9be9da8-0')

In [6]:
from crewai import Agent

researcher = Agent(
    role="Research Agent",
    goal="Research key facts, trends, and insights about the given topic",
    backstory="You are an expert content researcher who finds concise, reliable, and Instagram-relevant insights.",
    llm=llm,
    verbose=True,
    allow_delegation=False
)


/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:937: UserWarning: Mixing V1 models and V2 models (or constructs, like `TypeAdapter`) is not supported. Please upgrade `CrewAgentExecutor` to V2.
  warnings.warn(


In [7]:
writer = Agent(
    role="Content Writer",
    goal="Write engaging Instagram captions (short & long) with hashtags and CTA",
    backstory="You are a social media copywriter who creates viral Instagram content.",
    llm=llm,
    verbose=True,
    allow_delegation=False
)


In [8]:
reviewer = Agent(
    role="Reviewer",
    goal="Edit content for clarity, tone, grammar, and engagement",
    backstory="You are a senior editor ensuring Instagram-ready quality.",
    llm=llm,
    verbose=True,
    allow_delegation=False
)


In [9]:
image_prompt_agent = Agent(
    role="Image Prompt Generator",
    goal="Create detailed prompts for AI image generation",
    backstory="You are a creative designer skilled in writing prompts for text-to-image models.",
    llm=llm,
    verbose=True,
    allow_delegation=False
)


In [10]:
from crewai import Task

research_task = Task(
    description="""
    Research the topic: {topic}.
    Provide:
    - Key facts
    - Trends
    - 3–5 bullet insights
    - Simple language suitable for Instagram
    """,
    expected_output="Bullet-point research notes",
    agent=researcher
)


In [11]:
write_task = Task(
    description="""
    Using the research notes:
    - Write a SHORT Instagram caption (1–2 lines)
    - Write a LONG caption (5–7 lines)
    - Add CTA
    - Add 8–12 relevant hashtags
    """,
    expected_output="Instagram captions with hashtags and CTA",
    agent=writer
)


In [12]:
review_task = Task(
    description="""
    Review and improve the Instagram content:
    - Fix grammar
    - Improve engagement
    - Keep tone friendly and modern
    """,
    expected_output="Final polished Instagram content",
    agent=reviewer
)


In [13]:
image_prompt_task = Task(
    description="""
    Create 2–3 detailed image prompts for AI image generation.
    Prompts should be:
    - Instagram aesthetic
    - High quality
    - Relevant to the topic
    """,
    expected_output="2–3 detailed image prompts",
    agent=image_prompt_agent
)


In [14]:
from crewai import Crew

crew = Crew(
    agents=[researcher, writer, reviewer,image_prompt_agent],
    tasks=[research_task, write_task, review_task, image_prompt_task],
    verbose=2
)


In [15]:
topic = "AI in Healthcare"

result = crew.kickoff(inputs={"topic": topic})
print(result)


 [DEBUG]: == Working Agent: Research Agent
 [INFO]: == Starting Task: 
    Research the topic: AI in Healthcare.
    Provide:
    - Key facts
    - Trends
    - 3–5 bullet insights
    - Simple language suitable for Instagram
    


> Entering new CrewAgentExecutor chain...
I now can give a great answer.  
Final Answer: 

**Key Facts:**
- AI in healthcare is projected to reach $188 billion by 2030.
- Approximately 80% of healthcare executives believe that AI will improve patient outcomes.
- AI applications in healthcare include diagnostics, personalized medicine, and patient management.

**Trends:**
- Increasing investment in AI technologies by healthcare organizations.
- Growth of telemedicine and AI-driven virtual health assistants.
- Use of AI for predictive analytics to prevent diseases before they occur.

**Insights:**
- 🧠 AI can analyze medical data faster and more accurately than humans, helping doctors make better decisions.
- 💡 Personalized treatment plans are becoming more co

In [16]:
import re
import tiktoken

# result is already a string
result_str = result

# Extract prompts that start with a number and dot (1., 2., 3.)
prompts = re.findall(r'\d+\.\s+(.*?)(?=(\d+\.\s+|$))', result_str, re.DOTALL)
prompts = [p[0].replace('\n', ' ').strip() for p in prompts]

print("Extracted Prompts:")
for i, p in enumerate(prompts, 1):
    print(f"{i}: {p}\n")




Extracted Prompts:
1: **Prompt 1:** Create an Instagram aesthetic image depicting a futuristic healthcare setting where AI is actively being utilized. The scene should show a diverse team of healthcare professionals, including a doctor, nurse, and a data analyst, gathered around a high-tech holographic display that visualizes patient data and treatment plans. The background should feature sleek, modern medical equipment and soft, warm lighting to create a welcoming atmosphere. Include elements that signify AI, such as a digital brain or neural network graphics subtly integrated into the environment. The overall mood should be optimistic and innovative, highlighting the bright future of healthcare powered by AI.

2: **Prompt 2:** Generate a visually striking image of a virtual health assistant interacting with a patient in a cozy home setting. The virtual assistant should be represented as a friendly, glowing hologram projected from a smart device. The patient, an older adult, is sittin

In [17]:
from langchain_openai import ChatOpenAI
import tiktoken

tokenizer = tiktoken.get_encoding("cl100k_base")

def summarize_to_token_limit(prompt, llm, max_tokens=77):
    system_prompt = f"""
    You are an expert prompt optimizer for text-to-image models.
    Rewrite the following prompt so that:
    - ALL important visual details are preserved
    - Redundant words are removed
    - The final prompt is under {max_tokens} tokens
    - Style: concise, comma-separated, image-generation friendly
    - Do NOT add new details
    """

    response = llm.invoke([
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}
    ])

    compressed = response.content.strip()

    # Safety check
    tokens = tokenizer.encode(compressed)
    if len(tokens) > max_tokens:
        compressed = tokenizer.decode(tokens[:max_tokens])

    return compressed


In [18]:
compressed_prompts = [
    summarize_to_token_limit(p, llm, max_tokens=77)
    for p in prompts
]

for i, p in enumerate(compressed_prompts, 1):
    print(f"{i}: {p}\n")


1: Futuristic healthcare setting, diverse team of professionals (doctor, nurse, data analyst) around a high-tech holographic display visualizing patient data, sleek modern medical equipment, soft warm lighting, welcoming atmosphere, AI elements like digital brain or neural network graphics, optimistic and innovative mood, bright future of AI-powered healthcare.

2: Virtual health assistant as a glowing hologram from a smart device, interacting with an older adult on a cozy couch. Room decorated with plants and warm colors, conveying comfort and safety. Floating icons represent health metrics and reminders, illustrating personalized AI care. Evoke trust and reassurance, emphasizing AI's positive impact on patient engagement.

3: Vibrant outdoor health and wellness fair, diverse individuals engaging with AI technologies, booths for telemedicine, fitness apps, nutritional analysis, lively atmosphere, various ages and backgrounds, bright colors, banners, interactive displays, excitement ab

In [19]:
from diffusers import StableDiffusionPipeline
import torch
import os
from IPython.display import display
from PIL import Image

pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16
).to("cuda")

Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!


In [20]:
def generate_sd_diffusers(prompts, out_dir="sd_outputs_diffusers"):
    os.makedirs(out_dir, exist_ok=True)

    for i, prompt in enumerate(prompts, 1):
        print(f"\n🎨 Generating image {i}/{len(prompts)}")
        print(f"📝 Prompt: {prompt}\n")

        image = pipe(
            prompt,
            num_inference_steps=30,
            guidance_scale=7.5,
            height=768,      # 👈 Instagram-friendly
            width=768        # 👈 Instagram-friendly
        ).images[0]

        path = f"{out_dir}/image_{i}.png"
        image.save(path)
        display(image)

        print(f"✅ Saved: {path}")

In [21]:
STYLE_SUFFIX = ", ultra-detailed, cinematic lighting, clean aesthetic, high quality"
styled_prompts = [p + STYLE_SUFFIX for p in compressed_prompts]


In [1]:
generate_sd_diffusers(styled_prompts)


NameError: name 'generate_sd_diffusers' is not defined